Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [4]:
from itertools import product
from random import random, randint, shuffle, seed, choice
import numpy as np
from scipy import sparse

In [5]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points * 2654435761 + num_sets + density)
    sets = sparse.lil_array((num_sets, num_points), dtype = bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets - 1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [83]:
x = make_set_covering_problem(100, 100, .3)
print("Element at row=42 and column=42:", x[42, 42])

Element at row=42 and column=42: True


# Our Solution

# Set up

In [84]:
import numpy as np

from itertools import product
from random import random, randint, shuffle, seed, sample
from scipy import sparse
from copy import copy
from functools import reduce

In [85]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points * 2654435761 + num_sets + density)
    sets = sparse.lil_array((num_sets, num_points), dtype = bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets - 1), p] = True
    return sets

In [86]:
n = 5000
PROBLEM_SIZE = NUM_SETS = n
x = make_set_covering_problem(n, n, .3)

SETS = x.toarray()
print("Element at row=42 and column=42:", x[42, 42])

Element at row=42 and column=42: True


In [87]:
# create 10 random initial states to start the search from 

def make_random_initial_state(PROBLEM_SIZE):
    """Returns a random initial state"""
    return [choice([True,False]) for _ in range(PROBLEM_SIZE)] 


def make_random_initial_states(PROBLEM_SIZE, NUM_INITIAL_STATES):
    """Returns a list of random initial states"""
    return [make_random_initial_state(PROBLEM_SIZE=PROBLEM_SIZE) for _ in range(NUM_INITIAL_STATES)]

# create a random initial states
NUM_INITIAL_STATES = 10
initial_states = make_random_initial_states(PROBLEM_SIZE, NUM_INITIAL_STATES)


## Baseline

In [88]:
def tweak(state):
    new_state = copy(state)
    index = randint(0, PROBLEM_SIZE - 1)
    new_state[index] = not new_state[index]
    return new_state

def fitness2(state):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return valid, -cost

In [93]:
current_state = [False for _ in range(PROBLEM_SIZE)]
print(current_state)
print(fitness2(current_state))
count_fitness = 0
for step in range(1_000):
    new_state = tweak(current_state)
    if fitness2(new_state) > fitness2(current_state):
        current_state = new_state
        count_fitness += 1
        print(fitness2(current_state))
print(count_fitness)

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False

KeyboardInterrupt: 

In [ ]:
# multiple experiments with different initial states
for i, elem in enumerate(initial_states):
    print(f'Initial state {i}')
    current_state = copy(elem)
    #print(current_state)
    #print(fitness2(current_state))
    count_fitness = 0
    for step in range(10_000):
        new_state = tweak(current_state)
        if fitness2(new_state) > fitness2(current_state):
            current_state = new_state
            count_fitness += 1
            #print(fitness2(current_state))
    print("final state fitness ", fitness2(current_state))
    print("count_fitness: ", count_fitness)
    print("End of iteration number ", i)
    print()

### Random N-1 Tweaking

In [90]:
def tweak(state):
    new_state = copy(state)
    n_swapping = randint(0, PROBLEM_SIZE - 1)
    index_list = sample(range(0, PROBLEM_SIZE - 1), n_swapping)
    new_state = [not new_state[i] if i in index_list else new_state[i] for i in range(len(new_state))]
    return new_state

def fitness2(state):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return valid, -cost

In [91]:
current_state = [False for _ in range(PROBLEM_SIZE)]
print(current_state)
print(fitness2(current_state))
count_fitness = 0
for step in range(10_000):
    new_state = tweak(current_state)
    if fitness2(new_state) > fitness2(current_state):
        current_state = new_state
        print(fitness2(current_state))
        count_fitness += 1

print(count_fitness)

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False

KeyboardInterrupt: 

### Random 2 Tweak

In [94]:
def tweak(state):
    new_state = copy(state)
    index_list = sample(range(0, PROBLEM_SIZE - 1), 2)
    new_state = [not new_state[i] if i in index_list else new_state[i] for i in range(len(new_state))]
    return new_state

def fitness2(state):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return valid, -cost

In [95]:
current_state = [False for _ in range(PROBLEM_SIZE)]

print(fitness2(current_state))
count_fitness = 0
for step in range(1_000):
    new_state = tweak(current_state)
    
    if fitness2(new_state) > fitness2(current_state):
        current_state = new_state
        print(fitness2(current_state))
        count_fitness += 1

print(count_fitness)

(0, 0)
(2583, -2)
(3823, -4)
(4416, -6)
(4720, -8)
(4847, -10)
(4924, -12)
(4961, -14)
(4989, -16)
(4997, -18)
(4999, -20)
(5000, -22)
11


### Random 3 tweak

In [96]:
def tweak(state):
    new_state = copy(state)
    index_list = sample(range(0, PROBLEM_SIZE - 1), 3)
    new_state = [not new_state[i] if i in index_list else new_state[i] for i in range(len(new_state))]
    return new_state

def fitness2(state):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return valid, -cost

In [97]:
current_state = [False for _ in range(PROBLEM_SIZE)]

print(fitness2(current_state))
count_fitness = 0
for step in range(10_000):
    new_state = tweak(current_state)
    
    if fitness2(new_state) > fitness2(current_state):
        current_state = new_state
        print(fitness2(current_state))
        count_fitness += 1

print(count_fitness)

(0, 0)
(3251, -3)
(4375, -6)
(4815, -9)
(4938, -12)
(4981, -15)
(4993, -18)
(4998, -21)
(4999, -24)
(5000, -27)
9


### Simulated annealing

In [ ]:
def simulated_annealing(current_state, new_state, temperature):

    return np.exp(-(fitness2(current_state)[1] - fitness2(new_state)[1])/(temperature + 0.0001))

def tweak(state):
    new_state = copy(state)
    index_list = sample(range(0, PROBLEM_SIZE - 1), 2)
    new_state = [not new_state[i] if i in index_list else new_state[i] for i in range(len(new_state))]
    return new_state

def fitness2(state):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return valid, -cost


In [ ]:
current_state = [False for _ in range(PROBLEM_SIZE)]

temperature = 0.01
print(fitness2(current_state))
count_fitness = 0
for step in range(10_000):

    new_state = tweak(current_state)
    temperature = temperature * 0.5 if step % 100 == 0 else temperature

    if fitness2(new_state) > fitness2(current_state):

        current_state = new_state
        print(fitness2(current_state))
        count_fitness += 1
    
    else:
        
        p = simulated_annealing(current_state, new_state, temperature)
        
        if random() < p:
        
            current_state = new_state
            print("SA:", fitness2(current_state))
            count_fitness += 1

print(count_fitness)

# Steepest Ascent Hill Climbing (1 tweak)

In [98]:
def simulated_annealing(current_state, new_state, temperature):

    return np.exp(-(fitness2(current_state)[1] - fitness2(new_state)[1])/(temperature + 0.0001))

def tweak(state, index):
    new_state = copy(state)
    new_state[index] = not new_state[index]
    return new_state

def fitness2(state):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return valid, -cost


In [99]:
current_state = [False for _ in range(PROBLEM_SIZE)]
candidate_state = current_state

print(fitness2(current_state))
count_fitness = 0
best_index = 0
for step in range(1000):
    for i in range(PROBLEM_SIZE):
        new_state = tweak(current_state, i)
        if fitness2(new_state) > fitness2(candidate_state):
            candidate_state = new_state
            best_index = i
            count_fitness += 1
            print(step, count_fitness, fitness2(candidate_state))

    current_state = candidate_state   

print(fitness2(current_state))

print(count_fitness)

(0, 0)
0 1 (1491, -1)
0 2 (1524, -1)
0 3 (1528, -1)
0 4 (1531, -1)
0 5 (1607, -1)
0 6 (1613, -1)
1 7 (2625, -2)
1 8 (2636, -2)
1 9 (2649, -2)
1 10 (2671, -2)
1 11 (2695, -2)
1 12 (2723, -2)
2 13 (3385, -3)
2 14 (3417, -3)
2 15 (3421, -3)
2 16 (3423, -3)
2 17 (3468, -3)
2 18 (3478, -3)
2 19 (3479, -3)
2 20 (3480, -3)
3 21 (3923, -4)
3 22 (3951, -4)
3 23 (3954, -4)
3 24 (3959, -4)
3 25 (4002, -4)
3 26 (4012, -4)
4 27 (4295, -5)
4 28 (4312, -5)
4 29 (4324, -5)
4 30 (4327, -5)
4 31 (4335, -5)
4 32 (4337, -5)
4 33 (4356, -5)
4 34 (4357, -5)
5 35 (4533, -6)
5 36 (4549, -6)
5 37 (4574, -6)
5 38 (4577, -6)
5 39 (4583, -6)
5 40 (4584, -6)
5 41 (4585, -6)
5 42 (4589, -6)
5 43 (4593, -6)
6 44 (4711, -7)
6 45 (4723, -7)
6 46 (4733, -7)
6 47 (4736, -7)
6 48 (4737, -7)
6 49 (4738, -7)
6 50 (4740, -7)
6 51 (4742, -7)
6 52 (4743, -7)
6 53 (4744, -7)
6 54 (4752, -7)
7 55 (4826, -8)
7 56 (4828, -8)
7 57 (4836, -8)
7 58 (4839, -8)
7 59 (4841, -8)
7 60 (4843, -8)
7 61 (4849, -8)
7 62 (4850, -8)
7 63 (4851

KeyboardInterrupt: 

# Steepest Stochastic Ascent Hill Climbing (2 tweak)

In [100]:
def simulated_annealing(current_state, new_state, temperature):
    return np.exp(-(fitness2(current_state)[1] - fitness2(new_state)[1])/(temperature + 0.0001))

def tweak(state, index_list):
    new_state = copy(state)
    new_state = [not new_state[i] if i in index_list else new_state[i] for i in range(len(new_state))]
    return new_state

def fitness2(state):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return valid, -cost


In [101]:
current_state = [False for _ in range(PROBLEM_SIZE)]
candidate_state = current_state
star_state = None

print(fitness2(current_state))
count_fitness = 0
best_states = []
to_append = False

for step in range(1_000):
    candidate_state = current_state

    if missing_steps >= 5:
        if star_state is None or fitness2(star_state) > fitness2(star_state):
            star_state = copy(current_state)

        heavy_index_list = sample(range(0, PROBLEM_SIZE - 1), int(abs(fitness2(current_state)[1]/2)))
        print("Tried 5 times, no improvement...")
        current_state = [False for _ in range(PROBLEM_SIZE)]
        current_state = tweak(current_state, heavy_index_list) 
        candidate_state = current_state

        missing_steps = 0
        to_append = False

    for i in range(10_000):
        index_list = sample(range(0, PROBLEM_SIZE - 1), 2)
        new_state = tweak(current_state, index_list)
        if fitness2(new_state) > fitness2(candidate_state):
            missing_steps = 0
            candidate_state = new_state
            to_append = True
            print(step, fitness2(candidate_state))
        
    if current_state is not candidate_state:
        count_fitness += 1
        current_state = candidate_state
    else: 
        missing_steps += 1 

print("fitness_value:", fitness2(current_state)) if star_state is None else print("fitness_value:", fitness2(star_state))
print("count_fitness: ", count_fitness)

(0, 0)
0 (2551, -2)
0 (2563, -2)
0 (2605, -2)
0 (2613, -2)
0 (2618, -2)
0 (2622, -2)
0 (2627, -2)
0 (2655, -2)
0 (2656, -2)
0 (2667, -2)
0 (2668, -2)
0 (2683, -2)
0 (2690, -2)
1 (3833, -4)
1 (3904, -4)
1 (3932, -4)
1 (3948, -4)
1 (3962, -4)
1 (3966, -4)
2 (4499, -6)
2 (4504, -6)
2 (4516, -6)
2 (4521, -6)
2 (4525, -6)
2 (4541, -6)
2 (4546, -6)
2 (4550, -6)
2 (4552, -6)
3 (4787, -8)
3 (4797, -8)
3 (4812, -8)
3 (4814, -8)
3 (4815, -8)
4 (4899, -10)
4 (4911, -10)
4 (4913, -10)
4 (4924, -10)
4 (4925, -10)
4 (4929, -10)
4 (4930, -10)
4 (4931, -10)
4 (4933, -10)
4 (4934, -10)
5 (4962, -12)
5 (4965, -12)
5 (4977, -12)
5 (4978, -12)
5 (4980, -12)
5 (4981, -12)
5 (4982, -12)
5 (4985, -12)
6 (4991, -14)
6 (4994, -14)
6 (4998, -14)
6 (4999, -14)
7 (5000, -16)


KeyboardInterrupt: 